In [1]:
# !pip install ultralytics
# Install and update Ultralytics and Ray Tune packages
!pip install -U ultralytics "ray[tune]"

from IPython import display
display.clear_output()

from ultralytics import YOLO

from IPython.display import display, Image

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.6 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.6/107.7 GB disk)


In [2]:
import cv2
import torch
from torchvision.transforms import v2
from torchvision import transforms
from ultralytics import YOLO
from torchvision import models
import torch.nn as nn

In [3]:
!git clone -b Lam_Branch https://github.com/ntien-281/cs114-helmet-violation

Cloning into 'cs114-helmet-violation'...
remote: Enumerating objects: 6397, done.
remote: Counting objects: 100% (1692/1692), done.
remote: Compressing objects: 100% (1258/1258), done.
remote: Total 6397 (delta 167), reused 1676 (delta 162), pack-reused 4705
Receiving objects: 100% (6397/6397), 597.65 MiB | 28.89 MiB/s, done.
Resolving deltas: 100% (193/193), done.
Updating files: 100% (4956/4956), done.


In [4]:
org_transform = v2.Compose([
    v2.Resize((224, 224)),
    transforms.ToTensor()
])
def extract_shufflenet_class(shufflenet_output):
    _, predicted = torch.max(shufflenet_output, 1)

    return predicted.item()

def yolo_transform(frame):
    resized_image = cv2.resize(frame, (640,640))
    resized_image = transforms.ToTensor(resized_image)
    return resized_image

In [5]:
def calculate_iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2.tolist()
    # print('           ',x1, y1, w1, h1,'           ',x2, y2, w2, h2)
    # Calculate coordinates of the intersection
    x_intersection = max(x1, x2)
    y_intersection = max(y1, y2)
    w_intersection = min(x1 + w1, x2 + w2) - x_intersection
    h_intersection = min(y1 + h1, y2 + h2) - y_intersection

    # Check for no overlap
    if w_intersection <= 0 or h_intersection <= 0:
        return 0.0

    # Calculate areas of the bounding boxes
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    area_intersection = w_intersection * h_intersection

    # Calculate the IoU
    iou = area_intersection / (area_box1 + area_box2 - area_intersection)

    return iou

In [7]:
yol = YOLO('/content/cs114-helmet-violation/runs/detect/train/weights/best.pt')
shufflenet_model = models.shufflenet_v2_x1_0(pretrained=True)
shufflenet_model.fc = nn.Linear(shufflenet_model.fc.in_features, 2)
shufflenet_model.load_state_dict(torch.load('/content/cs114-helmet-violation/models/pretrained/shufflenet_finetuned.pth', map_location=torch.device('cpu')))
shufflenet_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth
100%|██████████| 8.79M/8.79M [00:00<00:00, 51.5MB/s]


ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [8]:
import os
import numpy as np

directory = os.fsencode('/content/cs114-helmet-violation/dataset/final_validate/images/test')
tp = np.zeros(0)
conf = np.zeros(0)
pred_cls = np.zeros(0)
target_cls = np.zeros(0)
fn = np.zeros(0)
for i, file in enumerate(os.listdir(directory)):
    filenamei = os.fsdecode(file)
    filenamet = filenamei[:-3] + 'txt'
    img = cv2.imread('/content/cs114-helmet-violation/dataset/final_validate/images/test/' + filenamei)
    yolo_pred = yol('/content/cs114-helmet-violation/dataset/final_validate/images/test/'+filenamei)[0].boxes
    con = yolo_pred.conf
    bb = yolo_pred.xywhn
    f = open('/content/cs114-helmet-violation/dataset/final_validate/labels/test/'+filenamet,'r')
    lines = f.readlines()
    detected_mark = np.array(range(0, len(lines)))
    for ib,pred_box in enumerate(bb):
        max_iou = 0
        max_box = []
        max_ind = 999
        max_label = 0
        x1, y1, x2, y2 = yolo_pred.xyxy[ib].tolist()
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        object_image = img[y1:y2, x1:x2]
        # cv2_imshow(object_image)
        shufflenet_input = org_transform(object_image)
        shufflenet_input = shufflenet_input.unsqueeze(0)
        with torch.no_grad():
          shufflenet_output = shufflenet_model(shufflenet_input)
        class_label = extract_shufflenet_class(shufflenet_output)
        # print('class_label:  ',class_label)
        for id,line in enumerate(lines):
          l, x, y, w, h = map(float, line.split())
          box = [x, y, w, h]
          if l == class_label:
            iou = calculate_iou(box, pred_box)
            if iou > max_iou and detected_mark[id] > -1:
              max_iou = iou
              max_box = box
              max_ind = id
              max_label = l
            if max_ind != 999 :
              detected_mark[max_ind] = -1

        if max_iou > 0.5:
            tp = np.append(tp, 1)
        else:
            tp = np.append(tp, 0)

        conf = np.append(conf, int(con[ib] * 100))
        pred_cls = np.append(pred_cls, class_label)
        target_cls = np.append(target_cls, max_label)

    for t in detected_mark[detected_mark > 0]:  #FN
      l, x, y, w, h = map(float, line.split())
      fn = np.append(fn, l)


image 1/1 /content/cs114-helmet-violation/dataset/final_validate/images/test/29765_jpg.rf.731cd8f4bbe21bf8cf85217244c647a7.jpg: 640x640 6 bikes, 432.2ms
Speed: 40.2ms preprocess, 432.2ms inference, 36.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/cs114-helmet-violation/dataset/final_validate/images/test/22158_jpg.rf.2642cfdf22a8b22ab3083299b3eaa5f1.jpg: 640x640 2 bikes, 231.1ms
Speed: 4.6ms preprocess, 231.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/cs114-helmet-violation/dataset/final_validate/images/test/22185_jpg.rf.fa298d5348a26c1f5cffffb2208507bd.jpg: 640x640 3 bikes, 228.6ms
Speed: 6.1ms preprocess, 228.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/cs114-helmet-violation/dataset/final_validate/images/test/28085_jpg.rf.1f172decb393f710affc4c93c0685d1d.jpg: 640x640 6 bikes, 229.8ms
Speed: 13.6ms preprocess, 229.8ms inference, 0.9ms postprocess per image at shape (1, 3,

In [9]:
import numpy as np

# from https://github.com/ultralytics/ultralytics
def compute_ap(recall, precision):
    """
    Compute the average precision (AP) given the recall and precision curves.

    Args:
        recall (list): The recall curve.
        precision (list): The precision curve.

    Returns:
        (float): Average precision.
        (np.ndarray): Precision envelope curve.
        (np.ndarray): Modified recall curve with sentinel values added at the beginning and end.
    """

    # Append sentinel values to beginning and end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([1.0], precision, [0.0]))

    # Compute the precision envelope
    mpre = np.flip(np.maximum.accumulate(np.flip(mpre)))

    # Integrate area under curve
    method = "interp"  # methods: 'continuous', 'interp'
    if method == "interp":
        x = np.linspace(0, 1, 101)  # 101-point interp (COCO)
        ap = np.trapz(np.interp(x, mrec, mpre), x)  # integrate
    else:  # 'continuous'
        i = np.where(mrec[1:] != mrec[:-1])[0]  # points where x-axis (recall) changes
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])  # area under curve

    return ap, mpre, mrec

def ap_per_class(tp, conf, pred_cls, target_cls, eps=1e-16):
    """
    Computes the average precision per class for object detection evaluation.

    Args:
        tp (np.ndarray): Binary array indicating whether the detection is correct (True) or not (False).
        conf (np.ndarray): Array of confidence scores of the detections.
        pred_cls (np.ndarray): Array of predicted classes of the detections.
        target_cls (np.ndarray): Array of true classes of the detections.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-16.

    Returns:
        ap (np.ndarray): Average precision for each class.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes, nt = np.unique(target_cls, return_counts=True)
    nc = unique_classes.shape[0]  # number of classes, number of detections

    # Create Precision-Recall curve and compute AP for each class
    x, prec_values = np.linspace(0, 1, 1000), []

    # Average precision
    ap = np.zeros((nc))

    for ci, c in enumerate(unique_classes):
        i = pred_cls == c
        # print('       i: ', i)
        # print('pred_cls: ', pred_cls)
        n_l = nt[ci]  # number of labels
        n_p = i.sum()  # number of predictions
        if n_p == 0 or n_l == 0:
            continue

        # Accumulate FPs and TPs
        fpc = (1 - tp[i]).cumsum(0)
        tpc = tp[i].cumsum(0)
        # Recall
        recall = tpc / (n_l + eps)  # recall curve

        # Precision
        precision = tpc / (tpc + fpc)  # precision curve

        # AP from recall-precision curve
        ap[ci], mpre, mrec = compute_ap(recall[:], precision[:])

    return ap
ap = ap_per_class(tp = tp,
             conf = conf,
             pred_cls = pred_cls,
             target_cls = target_cls)
print(ap)
print(np.mean(ap))

[    0.58307     0.81686]
0.6999659012163073
